#Overview & motivation 

All of our team members enjoy movies.  In addition to enjoying movies, we also enjoy working with API’s and somewhat structured data sets. Therefore, determining what makes a movie successful using the data available in the Internet Movie Database (IMDB) and Wikipedia seemed like a natural choice. 

In [6]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io

#Related Work 

#Initial Questions 

#Data 

In [40]:
# Deborah's function "find_movie"
# Will be used to make aadict

def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            pass
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                return None
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None

Pulling in Oscar CSV (Dana's Work)

In [317]:
# Read in Academy Awards df (AAdf)
AAdf = pd.read_excel("Academy_Awards_2006.xls")

# Update df
# Concat Original Title with first part of year; store in "title"
# This will be helpful when using ia.search_movie function
AAdf['Year'] = AAdf['Year'].values.astype(str)
AAdf['yr'] = AAdf.apply( lambda row: row['Year'][:4],axis=1 )
AAdf['titleyr'] = AAdf.apply( lambda row: '%s (%s)' % (row['Original Title'],row['Year'][:4]),axis=1)
# Convert 'Winner' "X" to 1
AAdf['Winner?'] = 1*(AAdf['Winner?'] == 'X')
## TO DO
### Convert 'Country' "0" to USA
# Convert 'Original Title' to be strings (easier for comparison later)
#AAdf['Original Title'] = str(AAdf['Original Title'])
# If movie title is "[no specific film title]", delete
AAdf = AAdf[AAdf['Original Title'] != "[no specific film title]"].copy()

# Store all possible awards in "awards" list
# This will be used when making aadict to indicate which awards the movie was nominated for/won
awards = list(set(list(AAdf['Category'])))

# Subset the data for easier use when making code
AAsubdf = AAdf[:12]
# View df
AAsubdf.head(5)

,Original Title,English Title,Sort Title,Year,Country,Award,Category,Winner?,Nominee(s),__ Academy Awards,Item,yr,titleyr
0,7th Heaven,7th Heaven,7th Heaven,1927-1928,0,Academy Award,Best Picture,0,Fox,1st,1,1927,7th Heaven (1927)
1,The Racket,The Racket,Racket,1927-1928,0,Academy Award,Best Picture,0,The Caddo Company,1st,2,1927,The Racket (1927)
2,Wings,Wings,Wings,1927-1928,0,Academy Award,Best Picture,1,Paramount Famous Lasky,1st,3,1927,Wings (1927)
3,Chang: A Drama of the Wilderness,Chang,Chang: A Drama of the Wilderness,1927-1928,0,Academy Award,Best Unique and Artistic Picture,0,Paramount Famous Lasky,1st,4,1927,Chang: A Drama of the Wilderness (1927)
4,The Crowd,The Crowd,Crowd,1927-1928,0,Academy Award,Best Unique and Artistic Picture,0,Metro-Goldwyn-Mayer,1st,5,1927,The Crowd (1927)


Note:  You can skip this cell and just run the next cell (pickle.load) to get the complete moviedict

In [20]:
x = AAdf
moviedict = dict.fromkeys(x['Original Title'])
for movie in moviedict:
    moviedict[movie] = {}
    moviedict[movie]['nominations'] = list(x[x['Original Title']==movie]['Category'])
    moviedict[movie]['won'] = list(x[(x['Original Title']==movie) & (x['Winner?']==1)]['Category'])
    moviedict[movie]['titleyr'] = list(x[x['Original Title']==movie]['titleyr'])[0]
    moviedict[movie]['year'] = list(x[x['Original Title']==movie]['Year'])[0]
    moviedict[movie]['nominees'] = list(x[x['Original Title']==movie]['Nominee(s)'])
    moviedict[movie]['titles'] = ia.search_movie(list(x[x['Original Title']==movie]['titleyr'])[0], _episodes=False)
    # For complete listing of movies...
    # ** Currently assuming the first movie in list is the correct one! **
    movieids = list()
    for i in range(0,len(moviedict[movie]['titles'])):
        movieids.append(moviedict[movie]['titles'][i].movieID)
    moviedict[movie]['movieIDs'] = movieids
    # Loop through awards list and indicate if movie was nominated or won
    for award in awards:
        moviedict[movie]["Nominated %s" % award] = award in list(moviedict[movie]['nominations'])
        moviedict[movie]["Won %s" % award] = award in list(moviedict[movie]['won'])
    # Only enter this loop if there was an imdb movieid found
    #if len(movieids) > 0:
        #mov = ia.get_movie(movieids[0])
        #moviedict[movie]['prodcomp'] = mov.data['production companies']
        #moviedict[movie]['cast'] = mov.data['cast']

In [439]:
%time

# Get unique list of movie titles from AAdf
AAuniquemovies = list(set(AAdf['Original Title']))

AAdict = {}
for i in range(0,len(AAuniquemovies)):
    
    ## Get arguments needed to search the movie title ##
    # movie title
    try:
        arg1 = AAuniquemovies[i]
        arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
        arg3 = ia.search_movie(str(arg1))  # list of possible movies
        movieobj = find_movie(str(arg1), arg2, arg3) # find the movie
    except ValueError:
        try:
            arg1 = str(AAuniquemovies[i])
            arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            movieobj = find_movie(arg1, arg2, arg3) # find the movie
        except ValueError:
            arg1 = AAuniquemovies[i]
            arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            movieobj = find_movie(arg1, arg2, arg3) # find the movie
    #arg1 = AAuniquemovies[i]  # movie title
    #arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
    #arg3 = ia.search_movie(arg1)  # list of possible movies
    # Could be that movie year is 1 year off (due to the Oscars being biennial in the beginning)
    if movieobj is None:
        arg2 = int(arg2) + 1
        movieobj = find_movie(arg1, arg2, arg3)
        #print movieobj is None
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
            #print movieobj is None
    print i, movieobj
    # Had to add this because some movies weren't getting any movies back ("Mighty Mouse in Gypsy Life" #247)
    if movieobj is not None:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['Original Title']==arg1]['Original Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['Original Title']==arg1]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['Original Title']==arg1) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['Original Title']==arg1]['Year'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['Original Title']==arg1]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        ## TO DO, list person/company associated with the award!!

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
0 Addams Family Values
1 Ernst & lyset
2 Complaints of a Dutiful Daughter
3 Z
4 Separate Tables
5 The Legend of Rockabye Point
6 9 (I)
7 Breaking Away
8 Blackboard Jungle
9 Rob Roy
10 Mike's New Car
11 Creature Comforts
12 Double or Nothing
13 The Tell Tale Heart
14 The Shop on Main Street
15 So You Think You're Not Guilty
16 Man of a Thousand Faces
17 Valmont
18 12:01 PM
19 Under Siege
20 Day for Night
21 The Absent-Minded Waiter
22 Hondo
23 Mulan
24 Ice-Capades
25 Monsieur Vincent
26 Once More, My Darling
27 The Divine Lady
28 That's Life!
29 Die gläserne Zelle
30 Every Day's a Holiday
31 Alice Adams
32 Air Force One
33 Liberators: Fighting on Two Fronts in World War II
34 Small Wonders
35 Lorenzo's Oil
36 Flying Down to Rio
37 Make a Wish
38 The Entertainer
39 Ghostbusters
40 The Mating Season
41 Is Paris Burning?
42 The Jaywalker
43 Kangaroo Court
44 Irma la Douce
45 It's a Mad, Mad, Mad, Mad World
46 Battle Cry
47 Wal

2015-11-24 00:42:26,925 CRITICAL [imdbpy] /Users/Dana/anaconda/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/find?q=The+Westerner&s=tt&mx=20', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', timeout('timed out',)), 'errmsg': 'timed out'},); kwds: {}
Traceback (most recent call last):
  File "/Users/Dana/anaconda/lib/python2.7/site-packages/imdb/parser/http/__init__.py", line 202, in retrieve_unicode
    uopener = self.open(url)
  File "/Users/Dana/anaconda/lib/python2.7/urllib.py", line 213, in open
    return getattr(self, name)(url)
  File "/Users/Dana/anaconda/lib/python2.7/urllib.py", line 350, in open_http
    h.endheaders(data)
  File "/Users/Dana/anaconda/lib/python2.7/httplib.py", line 1049, in endheaders
    self._send_output(message_body)
  File "/Users/Dana/anaconda/lib/python2.7/httplib.py", line 893, in _send_output
    self.se

 Songwriter


IMDbDataAccessError: {'exception type': 'IOError', 'url': 'http://akas.imdb.com/find?q=The+Westerner&s=tt&mx=20', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', timeout('timed out',)), 'errmsg': 'timed out'}

In [440]:
i = 538
try:
    arg1 = int(AAuniquemovies[i])
    arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
    arg3 = ia.search_movie(str(arg1))  # list of possible movies
    movieobj = find_movie(str(arg1), arg2, arg3) # find the movie
    print 1
except ValueError:
    try:
        arg1 = str(AAuniquemovies[i])
        arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
        arg3 = ia.search_movie(arg1)  # list of possible movies
        movieobj = find_movie(arg1, arg2, arg3) # find the movie
        print 2
    except ValueError:
        arg1 = AAuniquemovies[i]
        arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
        arg3 = ia.search_movie(arg1)  # list of possible movies
        movieobj = find_movie(arg1, arg2, arg3) # find the movie
        print 3
if movieobj is None:
    arg2 = int(arg2) + 1
    movieobj = find_movie(arg1, arg2, arg3)
    #print movieobj is None
    if (movieobj is None) & (arg3 != []):
        movieobj = arg3[0]
if movieobj is not None:
    print 1
else:
    print "hi"
print movieobj
#print arg1
#print arg2
#print arg3

2015-11-24 00:43:16,486 CRITICAL [imdbpy] /Users/Dana/anaconda/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/find?q=Songwriter&s=tt&mx=20', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', timeout('timed out',)), 'errmsg': 'timed out'},); kwds: {}
Traceback (most recent call last):
  File "/Users/Dana/anaconda/lib/python2.7/site-packages/imdb/parser/http/__init__.py", line 202, in retrieve_unicode
    uopener = self.open(url)
  File "/Users/Dana/anaconda/lib/python2.7/urllib.py", line 213, in open
    return getattr(self, name)(url)
  File "/Users/Dana/anaconda/lib/python2.7/urllib.py", line 350, in open_http
    h.endheaders(data)
  File "/Users/Dana/anaconda/lib/python2.7/httplib.py", line 1049, in endheaders
    self._send_output(message_body)
  File "/Users/Dana/anaconda/lib/python2.7/httplib.py", line 893, in _send_output
    self.send(

IMDbDataAccessError: {'exception type': 'IOError', 'url': 'http://akas.imdb.com/find?q=Songwriter&s=tt&mx=20', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', timeout('timed out',)), 'errmsg': 'timed out'}

In [444]:
i = 538
arg1 = str(AAuniquemovies[i])
arg2 = list(AAdf[AAdf['Original Title'] == AAuniquemovies[i]]['yr'])[0]  # movie year
#arg3 = ia.search_movie(arg1)  # list of possible movies
#movieobj = find_movie(arg1, arg2, arg3) # find the movie
print arg1, arg2

Songwriter 1984
2


In [415]:
AAdf[AAdf["Original Title"]=="Mighty Mouse in Gypsy Life"]

,Original Title,English Title,Sort Title,Year,Country,Award,Category,Winner?,Nominee(s),__ Academy Awards,Item,yr,titleyr
1894,Mighty Mouse in Gypsy Life,Mighty Mouse in Gypsy Life,Mighty Mouse in Gypsy Life,1945,0,Academy Award,"Best Short Subject, Cartoon",0,Paul Terry (Producer),18th,1895,1945,Mighty Mouse in Gypsy Life (1945)


In [381]:
#AAuniquemovies
#AAdf[AAdf['Original Title'] == 9]
#list(AAdf[AAdf['Original Title']==str(AAuniquemovies[20])]['Original Title'])[0]
#ia.search_movie(str('La Nuit Am\xe9ricaine'))

#try:
#    arg1 = int(AAuniquemovies[20])
#except ValueError:
#    try:
#        arg1 = str(AAuniquemovies[20])
#    except ValueError:
#        arg1 = AAuniquemovies[20]
#print type(arg1)
#arg1 = int(AAuniquemovies[6])
#print list(AAdf[AAdf['Original Title']==arg1]['Original Title'])[0]
#arg1 = str(AAuniquemovies[6])
AAuniquemovies[6]
#list(AAdf[AAdf['Original Title']==arg1]['Original Title'])[0]

9

In [292]:
pickle.dump(AAdict,open('moviedict','wb'))
#AAdict = pickle.load(open('moviedict','rb'))

For easier use in calling movies, change moviedict keys to be movieID
Changes from above:
1)  rename moviedict to aadict for "academy award dictionary"

In [20]:
aadict = dict.fromkeys(x['Original Title'])

In [9]:
filename = 'moviestemp.p'
movies = pickle.load(io.open(filename,'rb'))

In [222]:
a = ia.get_movie('5072808')
b = ia.get_movie('5072808')

In [30]:
a.isSameMovie(b)

1

In [10]:
movies

[<Movie id:5072808[http] title:_0 Degree C: A Love Story (2009)_>,
 <Movie id:4494824[http] title:_0 Feet Away (2014)_>,
 <Movie id:0244940[http] title:_0 Uhr 15, Zimmer 9 (1950)_>,
 <Movie id:3310038[http] title:_0+ omikron plus (2013)_>,
 <Movie id:1789663[http] title:_0,5 KM2 North (2010)_>,
 <Movie id:3692700[http] title:_0,60 mg (2014)_>,
 <Movie id:1042462[http] title:_0,7 ya! (2005)_>,
 <Movie id:2081142[http] title:_0.0270270 (2006)_>,
 <Movie id:2416424[http] title:_00 Schneider - Im Wendekreis der Eidechse (2013)_>,
 <Movie id:0109000[http] title:_00 Schneider - Jagd auf Nihil Baxter (1994)_>,
 <Movie id:4029848[http] title:_00-Baby Boom Child (2014)_>,
 <Movie id:0215459[http] title:_00/ciak operazione mondo (1966)_>,
 <Movie id:0057805[http] title:_002 operazione Luna (1965)_>,
 <Movie id:2183050[http] title:_003064.XZ (2012)_>,
 <Movie id:2141567[http] title:_006. un principio (2011)_>,
 <Movie id:5031210[http] title:_0068 Sniper's Nest (2015)_>,
 <Movie id:2158351[http] t

#Exploratory Data Analysis 

#Final Analysis

#Presentation 